## Raiders of the pottery GAN
### Usando Redes Generativas Adversarias para generación sintética de modelos 3D de materiales en arqueología

Los restos cerámicos proporcionan valiosa información que ayuda a datar el estrato arqueológico donde se encuentran, aportando <font color=violet>evidencias</font> de la producción local, relaciones comerciales, comportamiento de consumidores en poblaciones locales, etc. [1, 2]\cite{Orton93, Kampel2003}.

La <font color=violet>fragilidad de la cerámica</font> hace que la mayor parte del material recuperado en un yacimiento arqueológico no se corresponda con piezas completas por lo que la  <font color=violet>gran mayoría</font> del material disponible está  <font color=violet>fragmentado</font>.

<div style="text-align:center"><img src="https://media.giphy.com/media/uA147Pq6OvgCA/giphy.gif" /></div>

La colección está formada por <font color=violet>vasijas cerámicas de torno</font> encontradas en diversos yacimientos arqueológicos ibéricos ubicados en el Valle del Alto Gualquivir (España).

<div style="text-align:center"><img src="https://github.com/celiacintas/vasijas/blob/master/imagenes/dataset.png" width="400"/></div>


Está compuesta por unos miles de distintos recipientes, de los que se han registrado y clasificado <font color=violet>sus perfiles 2D proyectados</font> (vista lateral de cada una de ellas). La clasificación fue hecha en base a <font color=violet>criterios morfológicos</font> por un grupo de expertos~\cite{LucenaMTAP2014}.

## ¿Cómo generamos nuevas muestras 3D? 

<div style="text-align:center"><img src="https://github.com/celiacintas/vasijas/blob/master/imagenes/augmentation.png"/></div>



## ¿Qué bibliotecas de Python usamos para implementarnuestros experimentos?


<div style="text-align:center"><img src="https://media.giphy.com/media/JDmIrA84Pw1hK/giphy.gif" /></div>

In [ ]:
import ...